In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import h5py
from math import pi

from analysis.config import birds, h5_path_dict, pickle_dir
from analysis.ExpData import ExpData
from analysis.utils import get_fr

# Collect population patterns and save into pickle file
Visit patterns are (visits, neurs, frames) size

### Parameters

In [15]:
window = 30
fr_smoothing_window = int(20*0.5)
episode_significance = False

### Data formatting

In [16]:
data = {}
with open(pickle_dir / 'place_cells_1d.p', 'rb') as f: 
    episode_cells = pickle.load(f)
for key in h5_path_dict.keys():
    for fil in range(len(h5_path_dict[key])):
        fil_string = h5_path_dict[key][fil].as_posix()
        f = h5py.File(fil_string, 'r')
        exp_data = ExpData(f)
        data[fil_string] = {}
        cr_visits, noncr_visits = exp_data.get_cr_visits()
        c_visits = cr_visits[exp_data.cr_was_cache]
        r_visits = cr_visits[exp_data.cr_was_retrieval]
        hopcentered_visits = exp_data.get_hopcentered_visits(window)
        fr = get_fr(exp_data.spikes, fr_smoothing_window)
        visit_patterns = np.zeros(
            (hopcentered_visits.shape[0], exp_data.num_neurs, window*2 + 1)
            )
        for i, visit in enumerate(hopcentered_visits):
            visit_pattern = fr[:, visit]
            visit_pattern[:, visit == -1] = np.nan
            visit_patterns[i] = visit_pattern
        if episode_significance:
            significance = episode_cells[fil_string]['significance']
            visit_patterns = visit_patterns[:, significance > 0.99, :]
            print(np.sum(significance > 0.95))
        data[fil_string]['c_visits'] = c_visits
        data[fil_string]['r_visits'] = r_visits
        data[fil_string]['noncr_visits'] = noncr_visits
        data[fil_string]['fr'] = fr
        data[fil_string]['visit_patterns'] = visit_patterns
with open(pickle_dir / 'population_patterns.p', 'wb') as f: 
    pickle.dump(data, f)